In [6]:
%%capture --no-stderr
%pip install -U tavily-python
%pip install -U langchain_community
%pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.getenv("TAVILY_API_KEY"))

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool.invoke("What's a 'node' in LangGraph?")

[{'url': 'https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141',
  'content': 'Nodes: Nodes are the building blocks of your LangGraph. Each node represents a function or a computation step. You define nodes to perform specific tasks, such as processing input, making ...'},
 {'url': 'https://medium.com/@bhavikjikadara/langgraph-a-comprehensive-guide-for-beginners-ef17d3dd5383',
  'content': 'LangGraph is a library for building stateful, multi-actor applications with LLMs. ... We now need to define a few different nodes in our graph. In langgraph, a node can be either a function or a ...'}]

In [30]:
from typing import Annotated

from langchain_community.chat_models import ChatOllama
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [31]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [32]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]

In [33]:
local_llm =  "llama3-70b-8192" #"phi3:latest"
llm = ChatGroq(model=local_llm, temperature=0)
llm_with_tools = llm.bind_tools(tools)

In [34]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [35]:
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

In [50]:
graph = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])

In [52]:
config = {"configurable": {"thread_id": "1"}}

In [56]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    #{"messages": [("user", user_input)]}, config, stream_mode="values"
    None, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://langchain-ai.github.io/langgraph/", "content": "LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures ..."}, {"url": "https://github.com/langchain-ai/langgraph", "content": "LangGraph is a library for building stateful, multi-actor applications with LLMs using regular python functions or JS. Learn how to create graphs with nodes, edges, state, tools, and conditional edges for agentic behaviors."}]
================================== Ai Message ==================================

It seems like LangGraph is a library for building stateful, multi-actor applications with Large Langua

In [54]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

In [59]:
existing_message = snapshot.values["messages"][-2:]
existing_message

[HumanMessage(content="I'm learning LangGraph. Could you do some research on it for me?", id='b2caf6c0-a0d7-4bb2-bb33-c3070a43c67c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pepd', 'function': {'arguments': '{"query":"LangGraph"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 1055, 'total_tokens': 1103, 'completion_time': 0.137142857, 'prompt_time': 0.369147476, 'queue_time': None, 'total_time': 0.5062903329999999}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-be5b9058-bf52-4f65-b282-21867cb36737-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph'}, 'id': 'call_pepd'}], usage_metadata={'input_tokens': 1055, 'output_tokens': 48, 'total_tokens': 1103})]